In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
# load dependencies
import sys
import pickle
import os
import shutil
import json
import pandas as pd
import numpy as np
import rdkit.Chem as Chem
from rdkit.Chem import Crippen
from rdkit import DataStructs
from numpy.random import default_rng
import torch
from ast import literal_eval
from torch import nn, optim
from rdkit import Chem
from rdkit.Chem import AllChem
from tdc import Oracle
import subprocess

from rdkit.Chem import RDConfig
import os
import sys
sys.path.append(os.path.join(RDConfig.RDContribDir, 'SA_Score'))
# now you can import sascore!
import sascorer

In [3]:
print(os.getcwd())

/home/springnuance/reinvent-hitl/Base-Code-Binh


In [6]:
from training_Bradley_Terry_model.bradley_terry import BradleyTerryModel
from training_Rank_ListNet_model.rank_listnet import RankListNetModel
from training_Score_Regression_model.score_regression import ScoreRegressionModel
from scripts.helper import load_drd2_dataset, write_REINVENT_config, change_config_json, \
                    read_scaffold_result, load_feedback_model, smiles_human_score, \
                    compute_fingerprints, retrain_feedback_model,\
                    create_drd2_dataset, combine_drd2_dataset, save_drd2_dataset
                        
from sklearn.metrics import roc_curve, auc, precision_recall_fscore_support, matthews_corrcoef

def predict_proba_from_model_fps(feedback_type, feedback_model, features):

    # This is not computationally extensive yet
    # choose float 32
    
    features = torch.tensor(features, dtype=torch.float32)
    pred_label_proba = feedback_model.predict_proba(features).cpu().detach().numpy()
    return pred_label_proba


def check_create(path):
    """
    Check if the directory exists, if not, create it.
    """
    if not os.path.exists(path):
        os.makedirs(path)
    
def evaluate_results(
        output_dir, benchmark, # Choose whether to evaluate the results of the benchmark
        feedback_type, # scoring, comparing, ranking
        initial_feedback_model_path, # path to the initial feedback model
        final_testing_dataset_path, # path to the final training dataset
        model_pretrained_name, # Name of the pretrained model before REINVENT_round_1
        num_rounds, # number of rounds, corresponding to R in the paper
        num_iters, # number of iterations of showing molecules to the human for feedback at each round, corresponding to T in the paper
):
    """
    This function returns 
    - metrics: dictionary of metrics for each REINVENT round
    - drd2_proba: dictionary of DRD2 probabilities for each REINVENT round
    - novelty_score: dictionary of novelty scores for each REINVENT round
    - sa_score: dictionary of synthetic accessibility scores for each REINVENT round
    - qed_score: dictionary of QED scores for each REINVENT round
    - logP: dictionary of logP scores for each REINVENT round
    - mol_weight: dictionary of molecular weight scores for each REINVENT round
    - h_donors: dictionary of hydrogen bond donors for each REINVENT round
    - h_acceptors: dictionary of hydrogen bond acceptors for each REINVENT round
    - tpsa: dictionary of topological polar surface area for each REINVENT round
    - rotatable_bonds: dictionary of rotatable bonds for each REINVENT round
    - num_rings: dictionary of number of rings for each REINVENT round
    """
    
    final_testing_dataset = pd.read_csv(final_testing_dataset_path)

    print("Loading final testing dataset successfully")
        
    smiles_test = final_testing_dataset['smiles'].to_numpy()
    label_test = final_testing_dataset['label'].to_numpy()
    
    # for score regression, dataset_outputs have columns: smiles, features, label_proba, label_binary
    # for bradley-terry, dataset_outputs have columns: smiles_1, smiles_2, features_1, features_2, 
                                                      # label_1_proba, label_2_proba, label_1_binary, label_2_binary
                                                      # compare_proba, compare_binary
    # for rank listnet, dataset_outputs have columns: smiles_1/2/3, features_1/2/3, label_1/2/3_proba, label_1/2/3_binary
                                                     # label_1/2/3_softmax, label_1/2/3_rank
    
    ##########################
    # SAVING THE SMILES DATA #
    ##########################
    if benchmark["smiles_list"] == True:
        smiles_list_dict = {}
        for REINVENT_round in range(1, num_rounds + 2):
            reinvent_round_name = f"REINVENT_round_{REINVENT_round}"
            
            scaffold_memory_path = f"{output_dir}/{reinvent_round_name}/results/scaffold_memory.csv"
            
            if os.path.exists(scaffold_memory_path):
                scaffold_df = pd.read_csv(scaffold_memory_path)
                smiles_list_round = scaffold_df['SMILES'].to_numpy()
                smiles_list_dict[reinvent_round_name] = smiles_list_round
            else:
                print(f"Scaffold memory file {scaffold_memory_path} does not exist.")


    ##############
    # ML METRICS #
    ##############
    
    features_list = [compute_fingerprints(smiles) for smiles in smiles_test]
    features = np.array(features_list, dtype=np.float32)
    
    metrics = {}

    if benchmark["metrics"] == True:
        metrics_model = {
            'thresholds': [],
            'TP': [], 'TN': [], 'FP': [], 'FN': [],
            'accuracy': [], 'precision': [], 'recall': [], 'F1': [], 'MCC': [],
        }
        
        initial_feedback_model = load_feedback_model(feedback_type=feedback_type, 
                                                    feedback_model_path=initial_feedback_model_path)
        predicted_scores = predict_proba_from_model_fps(feedback_type, initial_feedback_model, features)

        # Compute metrics over a range of thresholds
        thresholds = np.linspace(0, 1, 101)
        for threshold in thresholds:
            predicted_labels = (predicted_scores > threshold).astype(int)
            TP = np.sum((predicted_labels == 1) & (label_test == 1))
            TN = np.sum((predicted_labels == 0) & (label_test == 0))
            FP = np.sum((predicted_labels == 1) & (label_test == 0))
            FN = np.sum((predicted_labels == 0) & (label_test == 1))

            accuracy = (TP + TN) / len(label_test)
            precision, recall, f1, _ = precision_recall_fscore_support(label_test, predicted_labels, average='binary')
            mcc = matthews_corrcoef(label_test, predicted_labels)

            metrics_model['thresholds'].append(threshold)
            metrics_model['TP'].append(TP)
            metrics_model['TN'].append(TN)
            metrics_model['FP'].append(FP)
            metrics_model['FN'].append(FN)
            metrics_model['accuracy'].append(accuracy)
            metrics_model['precision'].append(precision)
            metrics_model['recall'].append(recall)
            metrics_model['F1'].append(f1)
            metrics_model['MCC'].append(mcc)
        
        fpr, tpr, _ = roc_curve(label_test, predicted_scores, pos_label=1)
        roc_auc = auc(fpr, tpr)

        metrics_model['fpr'] = fpr
        metrics_model['tpr'] = tpr
        metrics_model['roc_auc'] = roc_auc
        metrics["REINVENT_round_1"] = metrics_model
        
        ########################
        # HITL MODEL BENCHMARK #
        ########################

        for REINVENT_round in range(2, num_rounds + 2):
            reinvent_round_name = f"REINVENT_round_{REINVENT_round}"
            
            hitl_iteration_name = f"HITL_iteration_{num_iters}"
            HITL_iteration = num_iters
            # for HITL_iteration in range(1, num_iters+1):
            #     hitl_iteration_name = f"HITL_iteration_{HITL_iteration}"    

            metrics_model = {
                'thresholds': [],
                'TP': [], 'TN': [], 'FP': [], 'FN': [],
                'accuracy': [], 'precision': [], 'recall': [], 'F1': [], 'MCC': [],
            }

            feedback_model_path = f"{output_dir}/REINVENT_round_{REINVENT_round - 1}/HITL_iteration_{HITL_iteration}/{model_pretrained_name}"

            if not os.path.exists(feedback_model_path):
                print(f"Feedback model {feedback_model_path} does not exist.")
            else:
                feedback_model = load_feedback_model(feedback_type=feedback_type, feedback_model_path=feedback_model_path)
                predicted_scores = predict_proba_from_model_fps(feedback_type, feedback_model, features)

                # Compute metrics over a range of thresholds
                thresholds = np.linspace(0, 1, 101)
                for threshold in thresholds:
                    predicted_labels = (predicted_scores > threshold).astype(int)
                    TP = np.sum((predicted_labels == 1) & (label_test == 1))
                    TN = np.sum((predicted_labels == 0) & (label_test == 0))
                    FP = np.sum((predicted_labels == 1) & (label_test == 0))
                    FN = np.sum((predicted_labels == 0) & (label_test == 1))
                    
                    accuracy = (TP + TN) / len(label_test)
                    precision, recall, f1, _ = precision_recall_fscore_support(label_test, predicted_labels, average='binary')
                    mcc = matthews_corrcoef(label_test, predicted_labels)
                    
                    metrics_model['thresholds'].append(threshold)
                    metrics_model['TP'].append(TP)
                    metrics_model['TN'].append(TN)
                    metrics_model['FP'].append(FP)
                    metrics_model['FN'].append(FN)
                    metrics_model['accuracy'].append(accuracy)
                    metrics_model['precision'].append(precision)
                    metrics_model['recall'].append(recall)
                    metrics_model['F1'].append(f1)
                    metrics_model['MCC'].append(mcc)
        
                # Compute ROC curve and AUC
                fpr, tpr, _ = roc_curve(label_test, predicted_scores, pos_label=1)
                roc_auc = auc(fpr, tpr)

                metrics_model['fpr'] = fpr
                metrics_model['tpr'] = tpr
                metrics_model['roc_auc'] = roc_auc

                #metrics[f"{reinvent_round_name}_{hitl_iteration_name}"] = metrics_model
                metrics[reinvent_round_name] = metrics_model
        
    #################################
    # DRD2 PROBABILITY DISTRIBUTION #
    #################################

    drd2_proba = {}

    if benchmark["drd2_proba"] == True:
        
        for REINVENT_round in range(1, num_rounds + 2):
            reinvent_round_name = f"REINVENT_round_{REINVENT_round}"
            
            scaffold_memory_path = f"{output_dir}/{reinvent_round_name}/results/scaffold_memory.csv"
            
            if os.path.exists(scaffold_memory_path):
                scaffold_df = pd.read_csv(scaffold_memory_path)
                smiles_list_round = scaffold_df['SMILES']
                
                # Load the TDC Oracle
                oracle = Oracle(name='DRD2')
                
                # Compute DRD2 probabilities
                drd2_probs = [oracle(smiles) for smiles in smiles_list_round]
                
                # Store the results
                drd2_proba[reinvent_round_name] = np.array(drd2_probs)
            else:
                print(f"Scaffold memory file {scaffold_memory_path} does not exist.")

    ##############################
    # ALL OTHER BENCHMARK SCORES #
    ##############################

    novelty_score = {}
    sa_score = {}
    qed_score = {}
    logP = {}
    mol_weight = {}
    h_donors = {}
    h_acceptors = {}
    tpsa = {}
    rotatable_bonds = {}
    num_rings = {}
    
    if benchmark["mol_props"] == True:
        for REINVENT_round in range(1, num_rounds + 2):
            reinvent_round_name = f"REINVENT_round_{REINVENT_round}"
            scaffold_memory_path = f"{output_dir}/{reinvent_round_name}/results/scaffold_memory.csv"
            
            if os.path.exists(scaffold_memory_path):
                scaffold_df = pd.read_csv(scaffold_memory_path)
                smiles_list = scaffold_df['SMILES']
                
                novelty_scores_round = []
                sa_scores_round = []
                qed_score_round = []
                logP_round = []
                mol_weight_round = []
                h_donors_round = []
                h_acceptors_round = []
                tpsa_round = []
                rotatable_bonds_round = []
                num_rings_round = []

                for smiles in smiles_list:
                    mol = Chem.MolFromSmiles(smiles)
                    if mol:
                        fps = compute_fingerprints(smiles)
                        # Reference fps is the testing features
                        similarities = DataStructs.BulkTanimotoSimilarity(fps, features_list)
                        max_similarity = max(similarities)
                        novelty = 1 - max_similarity
                        novelty_scores_round.append(novelty)
                        sa_scores_round.append(sascorer.calculateScore(mol))
                        qed_score_round.append(Chem.QED.qed(mol))
                        logP_round.append(Crippen.MolLogP(mol))
                        mol_weight_round.append(Chem.Descriptors.MolWt(mol))
                        h_donors_round.append(Chem.Descriptors.NumHDonors(mol))
                        h_acceptors_round.append(Chem.Descriptors.NumHAcceptors(mol))
                        tpsa_round.append(Chem.rdMolDescriptors.CalcTPSA(mol))
                        rotatable_bonds_round.append(Chem.rdMolDescriptors.CalcNumRotatableBonds(mol))
                        num_rings_round.append(Chem.rdMolDescriptors.CalcNumRings(mol))
                    else:
                        novelty_scores_round.append(None)
                        sa_scores_round.append(None)
                        qed_score_round.append(None)
                        logP_round.append(None)
                        mol_weight_round.append(None)
                        h_donors_round.append(None)
                        h_acceptors_round.append(None)
                        tpsa_round.append(None)
                        rotatable_bonds_round.append(None)
                        num_rings_round.append(None)   
                # Store the results
                novelty_score[reinvent_round_name] = np.array(novelty_scores_round)
                sa_score[reinvent_round_name] = np.array(sa_scores_round)
                qed_score[reinvent_round_name] = np.array(qed_score_round)
                logP[reinvent_round_name] = np.array(logP_round)
                mol_weight[reinvent_round_name] = np.array(mol_weight_round)
                h_donors[reinvent_round_name] = np.array(h_donors_round)
                h_acceptors[reinvent_round_name] = np.array(h_acceptors_round)
                tpsa[reinvent_round_name] = np.array(tpsa_round)
                rotatable_bonds[reinvent_round_name] = np.array(rotatable_bonds_round)
                num_rings[reinvent_round_name] = np.array(num_rings_round)
            else:
                print(f"Scaffold memory file {scaffold_memory_path} does not exist.")

    return smiles_list_dict, metrics, drd2_proba, novelty_score, sa_score, qed_score,\
            logP, mol_weight, h_donors, h_acceptors,\
            tpsa, rotatable_bonds, num_rings

### Running score regression model

In [7]:
feedback_type = "scoring" 

# feedback type as scoring:
# Given a molecule, what is the probability that the molecule is active regarding DRD2?  

num_rounds = 3 # number of rounds, corresponding to R in the paper
num_iters = 5 # number of iterations of showing molecules to the human for feedback at each round
num_queries = 56 # number of molecules, pairs or a set of molecules, dependig on the task, 
                 # shown to the simulated chemist at each HITL_iteration

for acquisition in ["random", "uncertainty", "greedy"]:
    for sigma_noise in [0.0, 0.1]:
        output_dir = f"output_score_regression_n_steps_100/R{num_rounds}_T{num_iters}_Q{num_queries}_acq_{acquisition}_noise_{sigma_noise}"
        initial_feedback_model_path = f"/home/springnuance/reinvent-hitl/Base-Code-Binh/training_Score_Regression_model/score_regression_model.pth"
        final_testing_dataset_path = "/home/springnuance/reinvent-hitl/Base-Code-Binh/data/drd2_final_test_large.csv"

        model_pretrained_name = "score_regression_model.pth"

        benchmark = {
            "smiles_list": True,
            "metrics": False,
            "drd2_proba": False,
            "mol_props": False,
        }

        smiles_list, metrics, drd2_proba, novelty_score, sa_score, qed_score,\
            logP, mol_weight, h_donors, h_acceptors,\
            tpsa, rotatable_bonds, num_rings = evaluate_results(
                output_dir, benchmark,
                feedback_type, # scoring, comparing, ranking
                initial_feedback_model_path,
                final_testing_dataset_path, # path to the final training dataset
                model_pretrained_name, # Name of the pretrained model before REINVENT_round_1
                num_rounds, # number of rounds, corresponding to R in the paper
                num_iters, # number of iterations of showing molecules to the human for feedback at each round, corresponding to T in the paper
        )

        np.save(f"results_score_regression_n_steps_100/acq_{acquisition}_noise_{sigma_noise}_smiles_list.npy", smiles_list)
        if benchmark["metrics"] == True:
            np.save(f"results_score_regression_n_steps_100/acq_{acquisition}_noise_{sigma_noise}_metrics.npy", metrics)
        if benchmark["drd2_proba"] == True:
            np.save(f"results_score_regression_n_steps_100/acq_{acquisition}_noise_{sigma_noise}_drd2_proba.npy", drd2_proba)
        if benchmark["mol_props"] == True:
            np.save(f"results_score_regression_n_steps_100/acq_{acquisition}_noise_{sigma_noise}_novelty_score.npy", novelty_score)
            np.save(f"results_score_regression_n_steps_100/acq_{acquisition}_noise_{sigma_noise}_sa_score.npy", sa_score)
            np.save(f"results_score_regression_n_steps_100/acq_{acquisition}_noise_{sigma_noise}_qed_score.npy", qed_score)
            np.save(f"results_score_regression_n_steps_100/acq_{acquisition}_noise_{sigma_noise}_logP.npy", logP)
            np.save(f"results_score_regression_n_steps_100/acq_{acquisition}_noise_{sigma_noise}_mol_weight.npy", mol_weight)
            np.save(f"results_score_regression_n_steps_100/acq_{acquisition}_noise_{sigma_noise}_h_donors.npy", h_donors)
            np.save(f"results_score_regression_n_steps_100/acq_{acquisition}_noise_{sigma_noise}_h_acceptors.npy", h_acceptors)
            np.save(f"results_score_regression_n_steps_100/acq_{acquisition}_noise_{sigma_noise}_tpsa.npy", tpsa)
            np.save(f"results_score_regression_n_steps_100/acq_{acquisition}_noise_{sigma_noise}_rotatable_bonds.npy", rotatable_bonds)
            np.save(f"results_score_regression_n_steps_100/acq_{acquisition}_noise_{sigma_noise}_num_rings.npy", num_rings)

Loading final testing dataset successfully
Loading final testing dataset successfully
Loading final testing dataset successfully
Loading final testing dataset successfully
Loading final testing dataset successfully
Loading final testing dataset successfully


### Running Bradley Terry model

In [8]:
feedback_type = "comparing" # scoring, comparing, ranking

# feedback type as comparing:
# Given two molecules, what is the probability that the first molecule is more active than the second molecule regarding DRD2?

num_rounds = 3 # number of rounds, corresponding to R in the paper
num_iters = 5 # number of iterations of showing molecules to the human for feedback at each round
num_queries = 8 # number of molecules, pairs or a set of molecules, dependig on the task, 
                 # shown to the simulated chemist at each HITL_iteration

for acquisition in ["random", "uncertainty", "greedy"]:
    for sigma_noise in [0.0, 0.1]:
        output_dir = f"output_bradley_terry_n_steps_100/R{num_rounds}_T{num_iters}_Q{num_queries}_acq_{acquisition}_noise_{sigma_noise}"
        initial_feedback_model_path = f"/home/springnuance/reinvent-hitl/Base-Code-Binh/training_Bradley_Terry_model/bradley_terry_model.pth"
        final_testing_dataset_path = "/home/springnuance/reinvent-hitl/Base-Code-Binh/data/drd2_final_test_large.csv"

        model_pretrained_name = "bradley_terry_model.pth"


        benchmark = {
            "smiles_list": True,
            "metrics": False,
            "drd2_proba": False,
            "mol_props": False,
        }

        smiles_list, metrics, drd2_proba, novelty_score, sa_score, qed_score,\
            logP, mol_weight, h_donors, h_acceptors,\
            tpsa, rotatable_bonds, num_rings = evaluate_results(
                output_dir, benchmark,
                feedback_type, # scoring, comparing, ranking
                initial_feedback_model_path,
                final_testing_dataset_path, # path to the final training dataset
                model_pretrained_name, # Name of the pretrained model before REINVENT_round_1
                num_rounds, # number of rounds, corresponding to R in the paper
                num_iters, # number of iterations of showing molecules to the human for feedback at each round, corresponding to T in the paper
        )

        np.save(f"results_bradley_terry_n_steps_100/acq_{acquisition}_noise_{sigma_noise}_smiles_list.npy", smiles_list)
        if benchmark["metrics"] == True:
            np.save(f"results_bradley_terry_n_steps_100/acq_{acquisition}_noise_{sigma_noise}_metrics.npy", metrics)
        if benchmark["drd2_proba"] == True:
            np.save(f"results_bradley_terry_n_steps_100/acq_{acquisition}_noise_{sigma_noise}_drd2_proba.npy", drd2_proba)
        if benchmark["mol_props"] == True:
            np.save(f"results_bradley_terry_n_steps_100/acq_{acquisition}_noise_{sigma_noise}_novelty_score.npy", novelty_score)
            np.save(f"results_bradley_terry_n_steps_100/acq_{acquisition}_noise_{sigma_noise}_sa_score.npy", sa_score)
            np.save(f"results_bradley_terry_n_steps_100/acq_{acquisition}_noise_{sigma_noise}_qed_score.npy", qed_score)
            np.save(f"results_bradley_terry_n_steps_100/acq_{acquisition}_noise_{sigma_noise}_logP.npy", logP)
            np.save(f"results_bradley_terry_n_steps_100/acq_{acquisition}_noise_{sigma_noise}_mol_weight.npy", mol_weight)
            np.save(f"results_bradley_terry_n_steps_100/acq_{acquisition}_noise_{sigma_noise}_h_donors.npy", h_donors)
            np.save(f"results_bradley_terry_n_steps_100/acq_{acquisition}_noise_{sigma_noise}_h_acceptors.npy", h_acceptors)
            np.save(f"results_bradley_terry_n_steps_100/acq_{acquisition}_noise_{sigma_noise}_tpsa.npy", tpsa)
            np.save(f"results_bradley_terry_n_steps_100/acq_{acquisition}_noise_{sigma_noise}_rotatable_bonds.npy", rotatable_bonds)
            np.save(f"results_bradley_terry_n_steps_100/acq_{acquisition}_noise_{sigma_noise}_num_rings.npy", num_rings)

Loading final testing dataset successfully
Loading final testing dataset successfully
Loading final testing dataset successfully
Loading final testing dataset successfully
Loading final testing dataset successfully
Loading final testing dataset successfully


### Running Rank ListNet model

In [9]:
feedback_type = "ranking" # scoring, comparing, ranking

# feedback type as ranking:
# Given N molecules, what are the orders of preference of these molecules regarding DRD2?

num_rounds = 3 # number of rounds, corresponding to R in the paper
num_iters = 5 # number of iterations of showing molecules to the human for feedback at each round
num_queries = 8 # number of molecules, pairs or a set of molecules, depending on the task, 
                 # shown to the simulated chemist at each HITL_iteration

for acquisition in ["random", "uncertainty", "greedy"]:
    for sigma_noise in [0.0, 0.1]:
        output_dir = f"output_rank_listnet_n_steps_100/R{num_rounds}_T{num_iters}_Q{num_queries}_acq_{acquisition}_noise_{sigma_noise}"
        initial_feedback_model_path = f"/home/springnuance/reinvent-hitl/Base-Code-Binh/training_Rank_ListNet_model/rank_listnet_model.pth"
        final_testing_dataset_path = "/home/springnuance/reinvent-hitl/Base-Code-Binh/data/drd2_final_test_large.csv"

        model_pretrained_name = "rank_listnet_model.pth"


        benchmark = {
            "smiles_list": True,
            "metrics": False,
            "drd2_proba": False,
            "mol_props": False,
        }

        smiles_list, metrics, drd2_proba, novelty_score, sa_score, qed_score,\
            logP, mol_weight, h_donors, h_acceptors,\
            tpsa, rotatable_bonds, num_rings = evaluate_results(
                output_dir, benchmark,
                feedback_type, # scoring, comparing, ranking
                initial_feedback_model_path,
                final_testing_dataset_path, # path to the final training dataset
                model_pretrained_name, # Name of the pretrained model before REINVENT_round_1
                num_rounds, # number of rounds, corresponding to R in the paper
                num_iters, # number of iterations of showing molecules to the human for feedback at each round, corresponding to T in the paper
        )

        np.save(f"results_rank_listnet_n_steps_100/acq_{acquisition}_noise_{sigma_noise}_smiles_list.npy", smiles_list)
        if benchmark["metrics"] == True:
            np.save(f"results_rank_listnet_n_steps_100/acq_{acquisition}_noise_{sigma_noise}_metrics.npy", metrics)
        if benchmark["drd2_proba"] == True:
            np.save(f"results_rank_listnet_n_steps_100/acq_{acquisition}_noise_{sigma_noise}_drd2_proba.npy", drd2_proba)
        if benchmark["mol_props"] == True:
            np.save(f"results_rank_listnet_n_steps_100/acq_{acquisition}_noise_{sigma_noise}_novelty_score.npy", novelty_score)
            np.save(f"results_rank_listnet_n_steps_100/acq_{acquisition}_noise_{sigma_noise}_sa_score.npy", sa_score)
            np.save(f"results_rank_listnet_n_steps_100/acq_{acquisition}_noise_{sigma_noise}_qed_score.npy", qed_score)
            np.save(f"results_rank_listnet_n_steps_100/acq_{acquisition}_noise_{sigma_noise}_logP.npy", logP)
            np.save(f"results_rank_listnet_n_steps_100/acq_{acquisition}_noise_{sigma_noise}_mol_weight.npy", mol_weight)
            np.save(f"results_rank_listnet_n_steps_100/acq_{acquisition}_noise_{sigma_noise}_h_donors.npy", h_donors)
            np.save(f"results_rank_listnet_n_steps_100/acq_{acquisition}_noise_{sigma_noise}_h_acceptors.npy", h_acceptors)
            np.save(f"results_rank_listnet_n_steps_100/acq_{acquisition}_noise_{sigma_noise}_tpsa.npy", tpsa)
            np.save(f"results_rank_listnet_n_steps_100/acq_{acquisition}_noise_{sigma_noise}_rotatable_bonds.npy", rotatable_bonds)
            np.save(f"results_rank_listnet_n_steps_100/acq_{acquisition}_noise_{sigma_noise}_num_rings.npy", num_rings)

Loading final testing dataset successfully
Loading final testing dataset successfully
Loading final testing dataset successfully
Loading final testing dataset successfully
Loading final testing dataset successfully
Loading final testing dataset successfully
